# Backdoor AI - Ollama on Google Colab

This notebook helps you run Ollama on Google Colab to use with your Backdoor AI application. You can install and run Ollama models (including Llama4) directly in Colab, then connect your Backdoor AI app to it.

## How it works

1. This notebook will install Ollama on this Colab instance
2. You'll select and download models (Llama4 or others)
3. We'll set up Cloudflared to create a secure tunnel to your Ollama instance
4. You'll get a URL to use in your Backdoor AI settings

Let's get started!

## 1. Set up environment

First, let's install the required packages. We'll need Ollama and Cloudflared for tunneling.

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Install cloudflared for tunneling
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# Install other dependencies
!pip install -q requests pyngrok httpx

# Set up directories
!mkdir -p /tmp/ollama/models

## 2. Start Ollama server

Now we'll start the Ollama server in the background.

In [ ]:
import subprocess
import time
import requests
import json
from IPython.display import clear_output

# Start Ollama server in background
ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for Ollama to start
print("Starting Ollama server...")
time.sleep(5)

# Check if Ollama is running
try:
    response = requests.get("http://localhost:11434/api/version")
    if response.status_code == 200:
        print(f"✅ Ollama is running! Version: {response.json().get('version')}")
    else:
        print(f"❌ Ollama returned unexpected status: {response.status_code}")
except Exception as e:
    print(f"❌ Failed to connect to Ollama: {e}")
    print("Trying to start again...")
    # Kill the previous process if it exists
    if ollama_process:
        ollama_process.terminate()
        time.sleep(2)
    # Try starting again
    !ollama serve &
    time.sleep(5)
    try:
        response = requests.get("http://localhost:11434/api/version")
        if response.status_code == 200:
            print(f"✅ Second attempt succeeded! Ollama is running. Version: {response.json().get('version')}")
    except:
        print("❌ Failed to start Ollama after multiple attempts.")

## 3. Choose and download a model

Now, let's download a model. We recommend Llama4 models for best performance or compatibility.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Define recommended models
recommended_models = [
    {"name": "Llama4 (70B - Best quality)", "id": "llama4:latest", "size": "~70GB"},
    {"name": "Llama4 (8B - Faster)", "id": "llama4-8b:latest", "size": "~8GB"},
    {"name": "Llama4 Code (Code-specialized)", "id": "llama4-code:latest", "size": "~70GB"},
    {"name": "Llama4 Tiny (Smallest, fastest)", "id": "llama4-tiny:latest", "size": "~1.5GB"},
    {"name": "Mistral (7B)", "id": "mistral:latest", "size": "~7GB"},
    {"name": "Gemma (7B)", "id": "gemma:latest", "size": "~7GB"},
    {"name": "Neural Chat (7B)", "id": "neural-chat:latest", "size": "~7GB"},
    {"name": "Custom model (enter below)", "id": "custom", "size": "varies"}
]

# Create dropdown for model selection
model_dropdown = widgets.Dropdown(
    options=[(f"{model['name']} ({model['size']})", model['id']) for model in recommended_models],
    description='Select model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Create text field for custom model
custom_model = widgets.Text(
    description='Custom model:',
    placeholder='Enter model name (e.g., llama4-7b:latest)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', display='none')
)

# Download button
download_button = widgets.Button(description='Download Model', button_style='primary')
output_area = widgets.Output()

# Function to handle model dropdown change
def on_model_change(change):
    if change['new'] == 'custom':
        custom_model.layout.display = 'block'
    else:
        custom_model.layout.display = 'none'

# Function to download model
def download_model(b):
    with output_area:
        clear_output()
        model_id = model_dropdown.value
        
        if model_id == 'custom':
            if not custom_model.value.strip():
                print("⚠️ Please enter a custom model name!")
                return
            model_id = custom_model.value.strip()
        
        print(f"🚀 Downloading model: {model_id}")
        print("This may take a while depending on the model size and your internet connection...")
        print("You'll see progress below. Please don't interrupt the process.")
        
        # Run ollama pull command
        process = subprocess.Popen(
            ["ollama", "pull", model_id],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        
        # Show live output
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                print(output.strip())
        
        return_code = process.poll()
        if return_code == 0:
            print(f"✅ Model {model_id} downloaded successfully!")
            # List available models
            print("\n📋 Available models:")
            !ollama list
        else:
            print(f"❌ Failed to download model {model_id}. Return code: {return_code}")

# Connect events
model_dropdown.observe(on_model_change, names='value')
download_button.on_click(download_model)

# Display widgets
display(widgets.HTML("<h3>Select a model to download:</h3>"))
display(model_dropdown)
display(custom_model)
display(download_button)
display(output_area)

## 4. Test the model

Let's make sure the model works by asking it a simple question.

In [ ]:
import requests
import json
import time
from IPython.display import display, HTML

# Function to test a model
def test_model(model_id, prompt="Hi, I'm testing if you're working properly. Please give a brief greeting."):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model_id,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            result = response.json()
            return {
                "success": True,
                "response": result.get("message", {}).get("content", "No content returned")
            }
        else:
            return {
                "success": False,
                "error": f"Server returned status code {response.status_code}: {response.text}"
            }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Get available models
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        model_options = [(model.get("name"), model.get("name")) for model in models]
    else:
        model_options = [("No models found", "")]
except Exception as e:
    model_options = [(f"Error: {str(e)}", "")]

# Create widgets
test_model_dropdown = widgets.Dropdown(
    options=model_options,
    description='Model to test:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

test_prompt = widgets.Textarea(
    value="Hi, I'm testing if you're working properly. Please give a brief greeting.",
    description='Prompt:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%', height='100px')
)

test_button = widgets.Button(description='Test Model', button_style='success')
test_output = widgets.Output()

# Function to test model
def on_test_button_click(b):
    with test_output:
        clear_output()
        if not test_model_dropdown.value:
            print("⚠️ Please select a model to test!")
            return
        
        print(f"🔍 Testing model '{test_model_dropdown.value}' with prompt: \n{test_prompt.value}\n")
        print("Waiting for response...")
        
        result = test_model(test_model_dropdown.value, test_prompt.value)
        
        if result["success"]:
            print("\n✅ Model responded successfully!\n")
            print("Response:")
            print("-----------------------------------")
            print(result["response"])
            print("-----------------------------------")
        else:
            print(f"\n❌ Error testing model: {result['error']}")

# Connect events
test_button.on_click(on_test_button_click)

# Display widgets
display(widgets.HTML("<h3>Test your model:</h3>"))
display(test_model_dropdown)
display(test_prompt)
display(test_button)
display(test_output)

## 5. Set up a tunnel to access your Ollama instance

Now we'll set up a Cloudflare tunnel so your Backdoor AI application can access this Ollama instance.

In [ ]:
import subprocess
import threading
import time
import re
from IPython.display import display, HTML

# Function to run cloudflared tunnel in a separate thread
def run_tunnel():
    global tunnel_process, tunnel_url
    tunnel_process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:11434"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    
    # Extract tunnel URL
    tunnel_url = None
    url_pattern = re.compile(r'https://[\w.-]+\.trycloudflare\.com')
    
    while True:
        line = tunnel_process.stdout.readline()
        if not line and tunnel_process.poll() is not None:
            break
        
        match = url_pattern.search(line)
        if match and not tunnel_url:
            tunnel_url = match.group(0)
            print(f"✅ Tunnel established at: {tunnel_url}")
            # Update tunnel info
            tunnel_info.value = f"<div style='padding: 10px; background-color: #e6ffe6; border-radius: 5px;'><b>✅ Your Ollama API is accessible at:</b><br><code>{tunnel_url}</code><br><br>Use this URL in your Backdoor AI settings as the Ollama API Base URL.<br>Keep this notebook running while you're using Ollama with your app!</div>"

# Create widgets for tunnel info
tunnel_button = widgets.Button(description='Start Tunnel', button_style='info')
tunnel_info = widgets.HTML("")

# Function to start tunnel
def on_tunnel_button_click(b):
    b.description = "Starting..."
    b.disabled = True
    tunnel_info.value = "<div style='padding: 10px; background-color: #fff3e6; border-radius: 5px;'>⏳ Creating secure tunnel to Ollama... (this may take a moment)</div>"
    
    # Start tunnel in separate thread
    thread = threading.Thread(target=run_tunnel)
    thread.daemon = True
    thread.start()
    
    # Check for tunnel URL
    attempts = 0
    while attempts < 10 and not tunnel_url:
        time.sleep(1)
        attempts += 1
    
    if not tunnel_url:
        tunnel_info.value = "<div style='padding: 10px; background-color: #ffe6e6; border-radius: 5px;'>❌ Failed to create tunnel. Check the output below for details.</div>"
        b.description = "Try Again"
        b.disabled = False

# Connect events
tunnel_button.on_click(on_tunnel_button_click)

# Initialize global variables
tunnel_process = None
tunnel_url = None

# Display widgets
display(widgets.HTML("<h3>Create a secure tunnel to your Ollama instance:</h3>"))
display(widgets.HTML("<p>This will create a public URL that you can use to connect your Backdoor AI application to this Ollama instance.</p>"))
display(tunnel_button)
display(tunnel_info)

## 6. Connect Backdoor AI to your Ollama instance

Once you have your tunnel URL, follow these steps to connect Backdoor AI to your Ollama instance:

1. Go to your Backdoor AI settings page
2. Select "Ollama (Remote)" as your LLM provider
3. In the "Ollama API URL" field, enter the tunnel URL from above
4. In the "Ollama Model" dropdown, select the model you downloaded
5. Click "Save Settings"

**Important Notes:**
- Keep this notebook running as long as you're using Ollama with your app
- The tunnel URL will change each time you restart this notebook
- Google Colab sessions have limited runtime (a few hours for free tier)
- Your model downloads will be lost when the Colab session ends

If you want a more permanent solution, consider setting up Ollama on your own machine or a cloud server.

## Additional Information

### Troubleshooting

If you encounter issues:

1. **Ollama not starting**: Try restarting the runtime (Runtime > Restart runtime) and run all cells again
2. **Model download failing**: Check your internet connection and try a smaller model
3. **Tunnel not working**: Make sure Ollama is running properly, then try starting the tunnel again

### Keeping Colab Active

Google Colab sessions will disconnect after periods of inactivity. To keep your session active:
- Keep the browser tab open
- Consider using browser extensions that simulate activity

### Shutting Down

When you're done, remember to:
1. Switch your Backdoor AI back to another provider if needed
2. Close this notebook

### For Production Use

For a more reliable solution, consider:
- Running Ollama on your own hardware or a cloud VM
- Setting up proper authentication and security measures
- Using persistent storage for model files